In [221]:
import glob
from dateutil import parser
import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pandasql as ps

In [222]:
def get_coinmarket():
    """Returns coin names, market caps, prices, and volume of all coins listed on coinmarketcap.com"""
    
    #scraping url
    url = "https://coinmarketcap.com/all/views/all/"
    
    #Getting coinmarketcap data
    response = requests.get(url)
    response_b = BeautifulSoup(response.content, "html.parser")
    
    #Getting coin names
    coins = [x.get_text() for x in response_b.find_all("a", attrs={"class":"currency-name-container link-secondary"})]
    
    #Getting market cap
    market_cap = [x["data-usd"] for x in response_b.find_all("td", attrs={"class":"no-wrap market-cap text-right"})]

    #Getting prices
    prices = [x["data-usd"] for x in response_b.find_all("a", attrs={"class":"price"})]
    
    #Getting volume
    volume = [x["data-usd"] for x in response_b.find_all("a", attrs={"class":"volume"})]
    
    #Getting current time
    time = [datetime.datetime.now()]*len(coins)
    
    #Merging into one dataframe
    coinmarket = pd.DataFrame({"Coins":coins, "Market Cap":market_cap, "Prices":prices, "Volume":volume, "Local Time":time})

    return coinmarket

def get_coin(filename):
    coins = pd.read_csv(filename).loc[:, "Coins":]    

    for column in coins.columns[1:]:
        coins[column] = pd.to_numeric(coins[column], errors="coerce")
        
    return coins

def get_times():
    coins = glob.glob("data/coin*")
    
    times = [parser.parse(x.split("_")[1][:-4]) for x in coins]
    
    return pd.DataFrame({"Coins":coins, "Times":times})

In [225]:
def five_minute_moving_average(coinmarket):
    
    #Getting elapsed time
    coinmarket["Elapsed"] = coinmarket["Local Time"].apply(lambda x: abs((datetime.datetime.now() - x).seconds))
    
    #Specifying for times within the past number of seconds
    coinmarket_window = ps.sqldf("""SELECT * FROM coinmarket WHERE Elapsed < 300""")
    print(coinmarket_window)
    
    #Getting average
    coinmarket_avg = ps.sqldf("""SELECT Coins, AVG(`Market Cap`) AS '5 Minute Average Market Cap', AVG(Prices) AS '5 Minute Average Prices', AVG(Volume) AS '5 Minute Average Volume' FROM coinmarket_window GROUP BY Coins""")
    
    #Dropping extraneous elapsed variable `Elapsed`
    return coinmarket_avg

In [226]:
coinmarket = get_coinmarket()

In [227]:
def scrape_job():
    
    #Getting current time
    curr = datetime.datetime.now()
    
    global coinmarket
    
    #Getting coin prices data
    coinmarket_upd = get_coinmarket()
    
    #Appending to current
    coinmarket = pd.concat([coinmarket, coinmarket_upd])
    
    #Saving progress
    coinmarket.to_csv("data/coin_" + str(curr) +  ".csv")
    
    #Getting five-minute moving average
    five_min = five_minute_moving_average(coinmarket)
    
    #Saving stats
    if not five_min.empty:
        five_min.to_csv("data/stats/coin_5min_" + str(curr) + ".csv")
        
    return None

In [228]:
from apscheduler.schedulers.blocking import BlockingScheduler

scheduler = BlockingScheduler()

scheduler.add_job(scrape_job, 'interval', minutes=1)

<Job (id=0e5f60803405491bae7f8721f55f9db1 name=scrape_job)>

In [ ]:
scheduler.start()

                 Coins                  Local Time     Market Cap  \
0              Bitcoin  2018-12-11 01:17:13.964345  61080159620.6   
1                  XRP  2018-12-11 01:17:13.964345  12642284261.6   
2             Ethereum  2018-12-11 01:17:13.964345  9612711793.03   
3              Stellar  2018-12-11 01:17:13.964345  2278895434.25   
4               Tether  2018-12-11 01:17:13.964345  1880218532.88   
5         Bitcoin Cash  2018-12-11 01:17:13.964345  1851066836.33   
6                  EOS  2018-12-11 01:17:13.964345   1743734629.9   
7           Bitcoin SV  2018-12-11 01:17:13.964345  1641784492.29   
8             Litecoin  2018-12-11 01:17:13.964345  1482913800.24   
9                 TRON  2018-12-11 01:17:13.964345  889533413.838   
10             Cardano  2018-12-11 01:17:13.964345  787130681.236   
11              Monero  2018-12-11 01:17:13.964345  740767851.958   
12                IOTA  2018-12-11 01:17:13.964345  690637060.277   
13                 NEM  2018-12-11

                 Coins  Elapsed                  Local Time     Market Cap  \
0              Bitcoin      199  2018-12-11 01:17:13.964345  61080159620.6   
1                  XRP      199  2018-12-11 01:17:13.964345  12642284261.6   
2             Ethereum      199  2018-12-11 01:17:13.964345  9612711793.03   
3              Stellar      199  2018-12-11 01:17:13.964345  2278895434.25   
4               Tether      199  2018-12-11 01:17:13.964345  1880218532.88   
5         Bitcoin Cash      199  2018-12-11 01:17:13.964345  1851066836.33   
6                  EOS      199  2018-12-11 01:17:13.964345   1743734629.9   
7           Bitcoin SV      199  2018-12-11 01:17:13.964345  1641784492.29   
8             Litecoin      199  2018-12-11 01:17:13.964345  1482913800.24   
9                 TRON      199  2018-12-11 01:17:13.964345  889533413.838   
10             Cardano      199  2018-12-11 01:17:13.964345  787130681.236   
11              Monero      199  2018-12-11 01:17:13.964345  740

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      239  2018-12-11 01:18:33.455230  61080159620.6   
1                   XRP      239  2018-12-11 01:18:33.455230  12642284261.6   
2              Ethereum      239  2018-12-11 01:18:33.455230  9612711793.03   
3               Stellar      239  2018-12-11 01:18:33.455230  2278895434.25   
4                Tether      239  2018-12-11 01:18:33.455230  1880218532.88   
5          Bitcoin Cash      239  2018-12-11 01:18:33.455230  1851066836.33   
6                   EOS      239  2018-12-11 01:18:33.455230   1743734629.9   
7            Bitcoin SV      239  2018-12-11 01:18:33.455230  1641784492.29   
8              Litecoin      239  2018-12-11 01:18:33.455230  1482913800.24   
9                  TRON      239  2018-12-11 01:18:33.455230  889533413.838   
10              Cardano      239  2018-12-11 01:18:33.455230  787130681.236   
11               Monero      239  2018-12-11 01:18:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      240  2018-12-11 01:20:33.298255  61080159620.6   
1                   XRP      240  2018-12-11 01:20:33.298255  12642284261.6   
2              Ethereum      240  2018-12-11 01:20:33.298255  9612711793.03   
3               Stellar      240  2018-12-11 01:20:33.298255  2278895434.25   
4                Tether      240  2018-12-11 01:20:33.298255  1880218532.88   
5          Bitcoin Cash      240  2018-12-11 01:20:33.298255  1851066836.33   
6                   EOS      240  2018-12-11 01:20:33.298255   1743734629.9   
7            Bitcoin SV      240  2018-12-11 01:20:33.298255  1641784492.29   
8              Litecoin      240  2018-12-11 01:20:33.298255  1482913800.24   
9                  TRON      240  2018-12-11 01:20:33.298255  889533413.838   
10              Cardano      240  2018-12-11 01:20:33.298255  787130681.236   
11               Monero      240  2018-12-11 01:20:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      240  2018-12-11 01:22:33.261491  61149319292.0   
1                   XRP      240  2018-12-11 01:22:33.261491  12632719680.3   
2              Ethereum      240  2018-12-11 01:22:33.261491  9622950090.29   
3               Stellar      240  2018-12-11 01:22:33.261491  2282770390.51   
4                Tether      240  2018-12-11 01:22:33.261491  1883925414.25   
5          Bitcoin Cash      240  2018-12-11 01:22:33.261491  1852878259.83   
6                   EOS      240  2018-12-11 01:22:33.261491   1741225593.6   
7            Bitcoin SV      240  2018-12-11 01:22:33.261491  1643630967.63   
8              Litecoin      240  2018-12-11 01:22:33.261491  1481017775.31   
9                  TRON      240  2018-12-11 01:22:33.261491  889560720.831   
10              Cardano      240  2018-12-11 01:22:33.261491  786852566.454   
11               Monero      240  2018-12-11 01:22:3

...

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      245  2018-12-11 02:18:33.003534  60850821417.8   
1                   XRP      245  2018-12-11 02:18:33.003534  12493969586.0   
2              Ethereum      245  2018-12-11 02:18:33.003534  9495951247.44   
3               Stellar      245  2018-12-11 02:18:33.003534  2254802926.85   
4                Tether      245  2018-12-11 02:18:33.003534  1883391025.73   
5          Bitcoin Cash      245  2018-12-11 02:18:33.003534   1845679767.6   
6                   EOS      245  2018-12-11 02:18:33.003534  1713898508.62   
7            Bitcoin SV      245  2018-12-11 02:18:33.003534  1614824599.68   
8              Litecoin      245  2018-12-11 02:18:33.003534  1466261385.98   
9                  TRON      245  2018-12-11 02:18:33.003534  888254899.322   
10              Cardano      245  2018-12-11 02:18:33.003534  776040603.427   
11               Monero      245  2018-12-11 02:18:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      245  2018-12-11 02:20:32.998491  60850821417.8   
1                   XRP      245  2018-12-11 02:20:32.998491  12493969586.0   
2              Ethereum      245  2018-12-11 02:20:32.998491  9495951247.44   
3               Stellar      245  2018-12-11 02:20:32.998491  2254802926.85   
4                Tether      245  2018-12-11 02:20:32.998491  1883391025.73   
5          Bitcoin Cash      245  2018-12-11 02:20:32.998491   1845679767.6   
6                   EOS      245  2018-12-11 02:20:32.998491  1713898508.62   
7            Bitcoin SV      245  2018-12-11 02:20:32.998491  1614824599.68   
8              Litecoin      245  2018-12-11 02:20:32.998491  1466261385.98   
9                  TRON      245  2018-12-11 02:20:32.998491  888254899.322   
10              Cardano      245  2018-12-11 02:20:32.998491  776040603.427   
11               Monero      245  2018-12-11 02:20:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      245  2018-12-11 02:22:33.287279  60669985652.2   
1                   XRP      245  2018-12-11 02:22:33.287279  12479743997.9   
2              Ethereum      245  2018-12-11 02:22:33.287279  9484718238.63   
3               Stellar      245  2018-12-11 02:22:33.287279  2252583704.53   
4                Tether      245  2018-12-11 02:22:33.287279  1882152790.91   
5          Bitcoin Cash      245  2018-12-11 02:22:33.287279  1801758725.88   
6                   EOS      245  2018-12-11 02:22:33.287279  1718830956.74   
7            Bitcoin SV      245  2018-12-11 02:22:33.287279  1611313938.49   
8              Litecoin      245  2018-12-11 02:22:33.287279  1463444757.32   
9                  TRON      245  2018-12-11 02:22:33.287279  887882674.113   
10              Cardano      245  2018-12-11 02:22:33.287279  774667168.711   
11               Monero      245  2018-12-11 02:22:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      246  2018-12-11 02:24:33.257856  60669985652.2   
1                   XRP      246  2018-12-11 02:24:33.257856  12479743997.9   
2              Ethereum      246  2018-12-11 02:24:33.257856  9484718238.63   
3               Stellar      246  2018-12-11 02:24:33.257856  2252583704.53   
4                Tether      246  2018-12-11 02:24:33.257856  1882152790.91   
5          Bitcoin Cash      246  2018-12-11 02:24:33.257856  1801758725.88   
6                   EOS      246  2018-12-11 02:24:33.257856  1718830956.74   
7            Bitcoin SV      246  2018-12-11 02:24:33.257856  1611313938.49   
8              Litecoin      246  2018-12-11 02:24:33.257856  1463444757.32   
9                  TRON      246  2018-12-11 02:24:33.257856  887882674.113   
10              Cardano      246  2018-12-11 02:24:33.257856  774667168.711   
11               Monero      246  2018-12-11 02:24:3

                  Coins  Elapsed                  Local Time     Market Cap  \
0               Bitcoin      246  2018-12-11 02:26:33.041246  60669985652.2   
1                   XRP      246  2018-12-11 02:26:33.041246  12479743997.9   
2              Ethereum      246  2018-12-11 02:26:33.041246  9484718238.63   
3               Stellar      246  2018-12-11 02:26:33.041246  2252583704.53   
4                Tether      246  2018-12-11 02:26:33.041246  1882152790.91   
5          Bitcoin Cash      246  2018-12-11 02:26:33.041246  1801758725.88   
6                   EOS      246  2018-12-11 02:26:33.041246  1718830956.74   
7            Bitcoin SV      246  2018-12-11 02:26:33.041246  1611313938.49   
8              Litecoin      246  2018-12-11 02:26:33.041246  1463444757.32   
9                  TRON      246  2018-12-11 02:26:33.041246  887882674.113   
10              Cardano      246  2018-12-11 02:26:33.041246  774667168.711   
11               Monero      246  2018-12-11 02:26:3

In [ ]:
five_min = five_minute_moving_average()

In [23]:
five_min

,Coins,Market Cap,Prices,Volume
0,Bitcoin,6.730895e+10,3867.160021,5.130715e+09
1,XRP,1.399054e+10,0.346924,3.910357e+08
2,Ethereum,1.115961e+10,107.738112,1.702785e+09
3,Stellar,2.852839e+09,0.148938,7.656440e+07
4,Bitcoin Cash,2.767826e+09,158.243198,6.991479e+07
5,EOS,2.333166e+09,2.574542,8.183312e+08
6,Tether,1.854323e+09,0.998869,3.205362e+09
7,Litecoin,1.831080e+09,30.817458,4.300982e+08
8,Bitcoin SV,1.640608e+09,93.867803,8.362543e+07
9,Cardano,9.889173e+08,0.038142,2.180510e+07


In [27]:
def get_5min(market):
    for column in market.columns[1:]:
        market[column + "_5minute"] = five_minute_moving_average(market[column])
    return market